In [ ]:
import csv
import secrets
from cryptography.fernet import Fernet

class FileAccessUtility:
    
    def __init__(self):
        self.admin_file = "admin.csv"
        self.users_file = "users.csv"
        self.accounts_file = "accounts.csv"
        # self.request_file = 'requests.csv'

    def get_admin_access_key(self):
        # Provide admin access key
        with open(self.admin_file, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                if row['term'] == 'admin_pass':
                    return row['secrets']
    
    def get_user_list(self):
        # provide user list
        user_list = []
        with open(self.users_file, 'r') as file:
            reader = csv.DictReader(file)
            for row in reader:
                user_list.append(row)
            return user_list
     
    def create_user(self, *args, **kwargs):
        # append an new user to file
        user = kwargs.get('user')
        del kwargs['user']
        if not user.is_authenticated and not user.is_admin:
            raise AdminOperationFailed("You are not allowed to perform this action")
        
        account_number = "BANK_0000"+secrets.token_hex(4)
        kwargs["id"] = str(secrets.token_hex(6))
        kwargs["account_number"] = account_number


        try:
            with open(self.users_file, "+a") as file:
                fieldnames = ["id","name", "email", "password", "role", "phone", "address", "is_active", 'account_number']
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writerow(kwargs)
                # Adding account information for customer user
                if kwargs['role'] == 'customer':

                    with open(self.accounts_file, '+a') as account_file:
                        fields_name = ['id', 'account_number', 'user_id', 'amount']
                        writer = csv.DictWriter(account_file, fieldnames=fields_name)
                        account_data = {
                            'id':secrets.token_hex(6),
                            'account_number':account_number,
                            'user_id':kwargs['id'],
                            'amount':0
                        }
                        writer.writerow(account_data)
                else:
                    # Leaving for handling different user roles. 
                    pass
        except Exception as e:
            print(e)

        
    def get_user(self, email):
        try:
            with open(self.users_file, 'r') as file:
                reader = csv.DictReader(file)
                for row in reader:
                    if row['email'] == email:
                        return row
                return None
        except:
            raise FileReadWriteError("Error While Reading User Data")
    
    def is_user_exists(self, email):
        user = self.get_user(email)
        if user:
            return True
        else:
            return False
    
    # Methods for Authentication
    def get_secret(self):
        # To get encryption key for enc/dec passwords
        try:
            with open(self.admin_file, 'r') as secret_file:
                reader = csv.DictReader(secret_file)
                for row in reader:
                    if row["term"] == "secret_key":
                        key_str = row['secrets']
                        key_bytes = key_str.strip("b'").strip("'").encode()
                        return key_bytes
        except:
            raise FileReadWriteError("Some error occurred while reading secret file")

    # Methods for Handling Accounts
    
    def get_accounts_data(self):
        print(self.accounts_file)
        try:
            with open(self.accounts_file, "r") as file:
                reader = csv.DictReader(file)
                account_data = [row for row in reader]
                return account_data  
        except Exception as e:
            print(e)
            print("Error while reading accounts.csv")
        
        
    def get_account_details(self, account_number):
        accounts = self.get_accounts_data()
        if len(accounts) != 0:
            for record in  accounts:
                if str(record['account_number']) == str(account_number):
                    return record
                else:
                    raise NoAccountExists("No record found.")
        else:
            raise NoAccountExists("Accounts File is empty.")

        
class FileReadWriteError(Exception):
    'raise when error while reading an writing file'
    pass
class AuthenticationError(Exception):
    "Raise when auth credentials not provided."
    pass

class AuthenticationFailedError(Exception):
    "Raise when auth failed incorrect credentials."
    pass

class AdminOperationFailed(Exception):
    "Raise when an non admin user try to perform admin operations."
    pass

class InsufficientBalance(Exception):
    'Raise when low balance'
    pass

class InvalidAmount(Exception):
    'Raise when enter invalid amount.'
    pass

class UserAlreadyExist(Exception):
    'Raise when duplicate user creation'
    pass

class NoAccountExists(Exception):
    'Raise when no account found'
    pass

# Authentication Class for handling user authentication
class Authentication(FileAccessUtility):
    @staticmethod
    def admin_operations(func):
        def wrapper(self, *args, **kwargs):
            user = kwargs.get('user')
            if user.is_authenticated and user.is_admin:
                result = func(user, *args, **kwargs)
                return result
            else:
                raise AdminOperationFailed("You are not allowed to perform this operation.")
        return wrapper

    def __init__(self):
        super().__init__()
        self.secret_key = self.get_secret()
        self.cipher = Fernet(self.secret_key)

    # Core Methods
    def get_secret(self):
        return super().get_secret()

    def get_binary_form(self, secret_term):
        return secret_term.strip("b'").strip("'").encode()


    def encrypt_password(self, secret_term):
        return self.cipher.encrypt(secret_term.encode())
    
    def decrypt_password(self, secret_term):
        secret_term = self.get_binary_form(secret_term)
        return self.cipher.decrypt(secret_term).decode()
    
    # Auth methods
    def authenticate_admin(self, secret):
        admin_stored_secrets = super().get_admin_access_key()
        if admin_stored_secrets:
            if secret == admin_stored_secrets:
                return True
            else:
                False
    
    # Specially for admin
    def create_user(self, *args, **kwargs): # user object property is_admin should true for access this method
        user_email = kwargs["email"]
        if self.is_user_exists(user_email):
            raise UserAlreadyExist(f"User Already Exist with email: {user_email}")
                
        kwargs['password'] = self.encrypt_password(kwargs['password'])
        print(kwargs)
        return super().create_user(*args, **kwargs)

    def login(self, email, password):
        stored_user = super().get_user(email=email)
        
        if stored_user:
            o_pass = self.decrypt_password(stored_user['password'])
            if stored_user['email'] == email and o_pass == password:
                return stored_user
            else:
                return None
        else:
            print("User Not Found")

# User 
class User:
    def __init__(self, email=None, password=None, admin_pass=None):
        if(not admin_pass):
            if(not email or not password):
                raise AuthenticationError("Credential were not provided")
            
        self.auth = Authentication()
        if admin_pass:
            print("Admin try to login.")
            admin_auth = self.auth.authenticate_admin(admin_pass)            
            self.is_admin = admin_auth # Can used in future
            self.role = "admin" if admin_auth else None
            self.is_authenticated = admin_auth
        else:
            print("User Try to login.")
            user = self.auth.login(email, password)
            # print(user)
            if not user:
                raise AuthenticationFailedError("Incorrect Credentials.")
            
            self.name = user.get('name') # if user else None
            self.email = user.get('email') # if user else None
            self.role = user.get('role') # if user else None
            self.phone = user.get('phone') # if user else None
            self.address = user.get('address') # if user else None
            self.is_active = user.get("is_active") # if user else None
            self.account_number = user.get("account_number")
            self.is_authenticated = True if user else False # user dict always should there.


class Account:
    def __init__(self, user):
        self.file_operations = FileAccessUtility()
        self.user = user # user refer to User class object
        account_details = self.file_operations.get_account_details(self.user.account_number)
        self.account_number = self.user.account_number
        self.amount = int(account_details.get('amount'))

    def deposit(self, deposit_amount):
        if deposit_amount < 1:
            raise InvalidAmount("Invalid deposit amount.")
        self.amount += deposit_amount

    def withdrawal(self, withdrawal_amount):
        if withdrawal_amount > self.amount:
            raise InsufficientBalance("Insufficient Balance")
        self.amount -= withdrawal_amount
    
    def check_balance(self):
        return self.amount
    
        
# Running code
try:
    user = User(admin_pass="77501692b45ced3ff57517b316bc731576ba9316f2ff06081f57a5c8282e0100")
    user_data = {
        "name":"Aysha Khaan",
        "email":"aysha@somebank.com", 
        'password':"user@1234",
        "role":"customer", 
        "phone":"1234567890", 
        "address":"123, Snajivani coloney", 
        "is_active":True,
        'user':user
    }
    user.auth.create_user(**user_data)
    aysha = User("aysha@somebank.com", "user@1234")
    aysha_account = Account(aysha)
    print("Your account balance is: Rs. ",aysha_account.check_balance())
    aysha_account.deposit(15000)
    print("Your account balance is: Rs. ",aysha_account.check_balance())
    aysha_account.withdrawal(5000)
    print("Your account balance is: Rs. ",aysha_account.check_balance())
    
    # Cause Error.
    aysha_account.deposit(0)
    aysha_account.withdrawal(15000)

    
except AuthenticationError as e:
    print(e)
except AuthenticationFailedError as e:
    print(e)
except AdminOperationFailed as e:
    print(e)
except InvalidAmount as e:
    print(e)
except InsufficientBalance as e:
    print(e)
except UserAlreadyExist as e:
    print(e)
except NoAccountExists as e:
    print(e)

User Try to login.
accounts.csv
Your account balance is: Rs.  0
Your account balance is: Rs.  15000
Your account balance is: Rs.  10000
Insufficient Balance


In [9]:
# auth = Authentication()
# fake_user = {
#     "name":"Shweta Pandita",
#     "email":"shaw@somebank.com", 
#     'password':"user@1234",
#     "role":"manager", 
#     "phone":"1234567890", 
#     "address":"123, park avenue", 
#     "is_active":True
# }
# auth.create_user(**fake_user)


# user = User("shaw@somebank.com", "user@1234")
# print(user.name)
# print(user.is_authenticated)
# print(user.role)

# auth.get_secret()
# auth.authenticate_admin("77501692b45ced3ff57517b316bc731576ba9316f2ff06081f57a5c8282e0100")
# auth = Authentication()
# fake_user = {
#     "name":"Shweta Pandita",
#     "email":"shaw@somebank.com", 
#     'password':"user@1234",
#     "role":"manager", 
#     "phone":"1234567890", 
#     "address":"123, park avenue", 
#     "is_active":True
# }
# auth.create_user(**fake_user)
# auth.login("shaw@somebank.com", "user@d1234")

accountss = [{'id': '6825a055b4b2', 'account_number': 'BANK_0000cfd62174', 'user_id': 'b4b7afd39525', 'amount': '0'}]
account_number = "BANK_0000cfd62174"
print([record for record in accountss if account_number == record['account_number']])
    

[{'id': '6825a055b4b2', 'account_number': 'BANK_0000cfd62174', 'user_id': 'b4b7afd39525', 'amount': '0'}]
